# Assemble the clusterization results

In [1]:
import os
import pickle

sents = []
for filename in os.listdir("sentences"):
    if filename.endswith(".pkl"):
        with open("sentences/"+filename,'rb') as f:
            sents.append(pickle.load(f))
            
            

# Load Conll data

In [2]:
import pandas

pdconll = pandas.read_pickle("Graph_clustering/pdconll_updated.pckl")

In [9]:
pdconll

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,sent_num,row_num,concomp,text,new_id,new_links
index,,,,,,,,,,,,,,,,,,,,
0,1,in,in.01,_,_,_,role=Location|fn=Temporal_collocation|id0=1|ha...,_,3,_,Location,_,_,_,0,0,1,0,1,3
0,2,vn:34_review,review.01,_,_,_,vncls=34|number=SG|id0=5|hasExtArg=yes|vn=vn:3...,_,"1,6",_,"Argument2,Argument1",_,_,_,0,1,1,0,2,"1,6"
0,3,attribute,attribute.01,_,_,_,manner=applied|clause_type=DECL|id0=45|hasExtA...,_,0,_,ROOT,_,_,_,0,2,1,0,3,-1
0,4,vn:11.3_take,take.01,_,_,_,vncls=11.3|fn=Capacity|id0=20|hasExtArg=yes|pl...,_,2,_,Argument2,_,_,_,0,3,1,0,4,2
0,5,misanthrope,misanthrope.01,_,_,_,number=SG|id0=9|hasExtArg=no|vn=misanthrope|pl...,_,2,_,Argument2,_,_,_,0,4,1,0,5,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39278,7,influence,influence.01,_,_,_,fn=Talking_into|id0=8|hasExtArg=yes|member=A1|...,_,5,_,Set,_,_,_,39278,685513,39265,1878,1176,1174
39278,8,bidder,bidder.01,_,_,_,number=SG|id0=16|hasExtArg=yes|vn=bidder|lex=b...,_,"11,6",_,"Argument2,Argument2",_,_,_,39278,685514,39265,1878,1177,"1180,1175"
39278,9,perhaps,perhaps.01,_,_,_,id0=12|hasExtArg=no|vn=perhaps|lex=perhaps_RB_...,_,0,_,ROOT,_,_,_,39278,685515,39265,1878,1178,-39265


# node type list, token vocabulary

In [8]:
ent_types = []
tok_vocab = {}
for _,row in pdconll.iterrows():
    ent_types += row[10].upper().split(",")
    w = row[2][:row[2].find(".")]
    if w not in tok_vocab:
        tok_vocab[w] = len(w)   
    
ent_types = set(ent_types)


# Generate CSV

In [ ]:
#for each text:
#1. Make "Title" from the first sentence
#For the remaining ones:
#2. Enumerate Entities;
#3. Enumerate <Entity Types>
#4. Enumerate E-Rel-E tuples;
#5. Generate the text of the sentence with <entitytype_N>
#6. Generate word_ids of the text with "-1"s instead of the entities

In [91]:
from tqdm import tqdm

#buid dict. {tok_id}->{tok_id}->rel_type
#+leave only one cluster per real sentence
def preprocessSents(pd,sents):
    print("Preprocessing..")
    #useful indexes
    sdict_inv = {}
    sdict_dir = {}
    for i,s in enumerate(sents):
        for t_id in s:
            if t_id not in sdict_inv:
                sdict_inv[t_id] = []
            sdict_inv[t_id].append(i)
            if i not in sdict_dir:
                sdict_dir[i] = []
            sdict_dir[i].append(t_id)
                
    #first pass. build real_sent->gen_sents dict
    rsent2sent = {}
    print ("First pass")
    with tqdm(total=len(pd)) as pbar: 
        for _,row in pd.iterrows():
            s_id = row['sent_num']
            if row['new_id'] in sdict_inv:
                cl_ids = sdict_inv[row['new_id']]
                if s_id not in rsent2sent:
                    rsent2sent[s_id] = {}
                for cl_id in cl_ids:    
                    if cl_id not in rsent2sent[s_id]:
                        rsent2sent[s_id][cl_id] = []
                    rsent2sent[s_id][cl_id].append(row['new_id'])
            pbar.update(1)
            
    #remove ambiguation. Just leave the biggest ones
    rsent2sent_filtered = {}
    for s_id in rsent2sent:
        if len(rsent2sent[s_id]) > 1:
            max_id = list(rsent2sent[s_id].keys())[0]
            for cl_id in rsent2sent[s_id]:
                if len(rsent2sent[s_id][cl_id]) > len(rsent2sent[s_id][max_id]):
                    max_id = cl_id
            rsent2sent_filtered[s_id] = rsent2sent[s_id][max_id]        
        else:
            rsent2sent_filtered[s_id] = list(rsent2sent[s_id].values())[0]
    
    #second pass. build dict 
    res_dict = {}
    rel_vocab = {}
    print ("Second pass")
    with tqdm(total=len(rsent2sent_filtered)) as pbar: 
        for s_id in rsent2sent_filtered:
            ents = rsent2sent_filtered[s_id]
            ents_set = set(ents)
            for e in ents:
                e_row = pd.loc[(pd['new_id'] == e)&(pd['sent_num']==s_id)]
                #print (e_row['new_links'].iloc[0])
                e_links = e_row['new_links'].iloc[0].split(",")

                for l in e_links:
                    if l in ents_set and l > "-1":
                        if e not in res_dict:
                            res_dict[e] = {}
                        l_row = pd.loc[(pd['new_id'] == l)&(pd['sent_num']==s_id)]
                        link_type = l_row[10].iloc[0] + "_" + e_row[10].iloc[0]
                        if link_type not in rel_vocab:
                            rel_vocab[link_type] = len(rel_vocab)

                        res_dict[e][l] = link_type
            pbar.update(1) 
            #break
    return res_dict, rel_vocab
    

def genIds(txt,tok_vocab):
    return " ".join([str(tok_vocab.get(t,"-1")) for t in txt])
        
def process_conll(pd,rel_vocab,tok_vocab,sents):
    cur_sent_num = None
    cur_txt_num = None

    collect_title = False
    title = []
    txt = []
    rels = []
    ents = {}
    ent_strs = []
    ent2num = {}
    ent_types = []
    
    datas = []
    print("Building...")

    with tqdm(total=len(pd)) as pbar: 
        for _,row in pd.iterrows():
            text_num = row["text"]
            sent_num = row["sent_num"]
            if cur_sent_num != None and sent_num != cur_sent_num:
                #Generate line for the output
                if not collect_title:
                    if len(ent2num) > 0:
                        processed = {}
                        for sid in list(ent2num.keys()):
                            sentence = sents[sid]
                            if sid in ent2num:
                                if sid not in processed:
                                    processed[sid] = True
                                    for s in sentence:
                                        if s in ent2num:
                                            processed[s] = True
                                            #print(sid,s,ent2num)
                                            rels.append(str(ent2num[sid]) + " " + str(rel_vocab[sentence[s]]) + " " + str(ent2num[s]))

                    
                    if len(rels) > 0:
                        tmp = []
                        tmp.append(" ".join(title))
                        tmp.append(";".join(ent_strs))
                        tmp.append(" ".join(ent_types))
                        tmp.append(";".join(rels))
                        tmp.append(" ".join(txt))
                        tmp.append(genIds(txt,tok_vocab))
                        datas.append(tmp)
        
                    txt = []
                    rels = []
                    ents = {}
                    ent_strs = []
                    ent2num = {}
                    ent_types = [] 
                else:
                    collect_title  = False

                cur_sent_num = sent_num
            else:    
                if text_num != cur_txt_num:
                    cur_txt_num = text_num 
                    collect_title = True
                    cur_sent_num = sent_num
                    title = []

                if collect_title:
                    #collect first sentence as a title
                    if row[2].find(".") == -1:
                        title.append(row[1])
                    else:    
                        title.append(row[2][:row[2].find(".")])                    
                else:
                    tok_id = row['new_id']
                    #if it is entity - add it
                    if tok_id in sents:
                        #remove duplicates, replace "," to "_"
                        ent_type = "_".join(list(set(row[10].split(","))))
                        ent_str = "<" + ent_type + ">"
                        if ent_str not in ents:
                            ents[ent_str] = []
                        ents[ent_str] = tok_id   
                        ent2num[tok_id] = len(ent2num)
                        if row[2].find(".") == -1:
                            ent_strs.append(row[1])
                        else:    
                            ent_strs.append(row[2][:row[2].find(".")])
                        ent_types.append(ent_str)
                        txt.append("<" + ent_type + "_"+ str(len(ents)) +">")
                    else:
                        if row[2].find(".") == -1:
                            txt.append(row[1])
                        else:    
                            txt.append(row[2][:row[2].find(".")])

            pbar.update(1)    
            #print (collect_title)
            
    return pandas.DataFrame(datas),datas            
             

In [14]:
dsents, rel_vocab = preprocessSents(pdconll,sents)


  0%|          | 0/685518 [00:00<?, ?it/s]

Preprocessing..
First pass


  0%|          | 0/34487 [00:00<?, ?it/s]

Second pass


100%|██████████| 34487/34487 [20:53:56<00:00,  2.18s/it]         


In [92]:
gw_dataset, datas = process_conll(pdconll,rel_vocab,tok_vocab,dsents)

  0%|          | 1/685518 [00:00<150:45:22,  1.26it/s]

Building...


100%|██████████| 685518/685518 [02:18<00:00, 4962.84it/s]


In [93]:
gw_dataset

,0,1,2,3,4,5
0,in review attribute take misanthrope 19 to rol...,Haag;Elianti,<Argument1> <Argument2>,0 9 0;1 84 1,<Argument1_1> play <Argument2_2>,-1 4 -1
1,car expect Royce Motor Rolls Inc. - it remain ...,"auto;sell;luxury;in;year;1,214;U.S.;last",<Elaboration> <ROOT> <Elaboration> <Location> ...,0 81 2;3 18 4;3 43 5;3 410 7;3 86 6,<Elaboration_1> <ROOT_2> <Elaboration_2> car <...,-1 -1 -1 3 -1 -1 -1 -1 -1
2,car expect Royce Motor Rolls Inc. - it remain ...,Mosher;and;officer;president;anticipate;execut...,<Argument1> <Elaboration> <Set> <Set> <Argumen...,0 544 1;2 1043 8;2 10 3;2 168 4;2 40 7;2 2 6;2...,<Argument1_1> <Elaboration_2> say <Set_3> <Set...,-1 -1 3 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 3 ...
3,industry Inc. increase quarterly cent cent its...,new;payable;15;Feb.,<ROOT> <ROOT> <Time> <Elaboration>,0 79 3;0 44 2;0 259 1,<ROOT_1> <ROOT_1> <Time_2> <Elaboration_3>,-1 -1 -1 -1
4,industry Inc. increase quarterly cent cent its...,record;set;not,<Elaboration> <Argument1> <ROOT>,0 350 2;0 1094 1;0 38 0,<Elaboration_1> <Argument1_2> <ROOT_3>,-1 -1 -1
...,...,...,...,...,...,...
30558,Trump withdraw face offer before doubt $ his c...,addition;break;coalition;apart;now;of;and;mana...,<Argument2> <ROOT> <Argument1> <NAME> <Time> <...,0 22 2;0 37 3;0 43 5;0 45 4;0 153 1;6 18 5;6 1...,<Argument2_1> <ROOT_2> <Argument1_3> <NAME_4> ...,-1 -1 -1 -1 -1 -1 -1 -1 -1 -1 8 4 -1 -1 -1 -1 ...
30559,Trump withdraw face offer before doubt $ his c...,its;leader;outside;in;union;30;top;Chicago;mee...,<ROOT> <Argument1> <Location> <Location> <Elab...,1 70 5;1 17 3;1 17 4;1 325 2;1 70 6;7 39 5;7 1...,resilience 's get today test convene first <RO...,10 2 3 5 4 7 5 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 8 10
30560,Trump withdraw face offer before doubt $ his c...,member;overwhelmingly;why;explain;last;task;to...,<Argument1> <Manner> <Purpose> <Argument2> <RO...,1 263 2;1 17 7;1 79 6;1 1440 3;1 81 4;1 17 5;1...,out finance approve banks refuse week <Argumen...,3 7 7 5 6 4 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -...
30561,Trump withdraw face offer before doubt $ his c...,vow;pursue;decide;acquisition;whatever,<ROOT> <Argument2> <Elaboration> <Argument2> <...,0 33 1;0 18 2;0 65 4;0 43 3,pilot <ROOT_1> <Argument2_2> <Elaboration_3> <...,5 -1 -1 -1 -1 5 -1


In [94]:
from sklearn.model_selection import train_test_split

gw_dataset_train, gw_dataset_test = train_test_split(gw_dataset, test_size=0.2, random_state=1)

gw_dataset_train, gw_dataset_val = train_test_split(gw_dataset_train, test_size=0.25, random_state=1)

In [101]:
gw_dataset_train.to_csv("conll_train.tsv",header=False,index=False,sep="\t")
gw_dataset_test.to_csv("conll_test.tsv",header=False,index=False,sep="\t")
gw_dataset_val.to_csv("conll_val.tsv",header=False,index=False,sep="\t")

with open("relations.vocab","w") as f:
    for r in rel_vocab:
        f.write(r+"\n")


In [87]:
et = []
et2 = []
for e in ent_types:
    et.append("<"+e+">")
    et2.append(e)

In [88]:
et

['<ROOT>',
 '<DIRECTION>',
 '<BENEFACTIVE>',
 '<ARGUMENT1>',
 '<ARGUMENT2>',
 '<ARGUMENT6>',
 '<ARGUMENT10>',
 '<MANNER>',
 '<EXTENT>',
 '<ELABORATION>',
 '<SET>',
 '<ARGUMENT3>',
 '<LOCATION>',
 '<ARGUMENT4>',
 '<NAME>',
 '<PURPOSE>',
 '<TIME>',
 '<NONCORE>']

In [89]:
et2

['ROOT',
 'DIRECTION',
 'BENEFACTIVE',
 'ARGUMENT1',
 'ARGUMENT2',
 'ARGUMENT6',
 'ARGUMENT10',
 'MANNER',
 'EXTENT',
 'ELABORATION',
 'SET',
 'ARGUMENT3',
 'LOCATION',
 'ARGUMENT4',
 'NAME',
 'PURPOSE',
 'TIME',
 'NONCORE']